In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from petsc4py import PETSc

%matplotlib inline

import conduction
help(conduction)

In [ ]:
minX, maxX = 0., 1.
minY, maxY = 0., 2.
minZ, maxZ = 0., 3.

nx, ny, nz = 20, 20, 20

mesh = conduction.Conduction3D((minX, minY, minZ), (maxX, maxY, maxZ), (nx, ny, nz))

In [ ]:
def z_function(z):
    return 1.0 - np.exp(z)

# np.column_stack([z_function(mesh.coords[:,2]), mesh.coords[:,2]])

# mesh.refine(z_fn=z_function)

## Update material properties

In [ ]:
coords = mesh.coords

nx = mesh.nx
ny = mesh.ny
nz = mesh.nz

k = np.ones(nz*ny*nx)
H = np.ones(nz*ny*nx)
# H.fill(0.)

mask = coords[:,0] >= 0.5

k[mask] = 5.

mesh.update_properties(k, H)

##  Boundary conditions

In [ ]:
mesh.boundary_condition('minZ', 1.0, flux=True)
mesh.boundary_condition('maxZ', 0.0, flux=False)

In [ ]:
sol = mesh.solve()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(111, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY),)
sca = ax1.scatter3D(coords[:,0], coords[:,1], coords[:,2], c=sol)
fig.colorbar(sca)

In [ ]:
H5_filname = 'rectilinear.h5'

mesh.save_mesh_to_hdf5(H5_filname)
mesh.save_field_to_hdf5(H5_filname, T=sol)

conduction.tools.generateXdmf(H5_filname)

In [ ]:
from time import clock

mat = mesh.construct_matrix()
rhs = mesh.construct_rhs()
sol = mesh.dm.createGlobalVector()


t = clock()
# manual solver

ksp = PETSc.KSP().create()
ksp.setType('cgs')
ksp.setOperators(mat)
# pc = ksp.getPC()
# pc.setType('gamg')
# pc.setGAMGLevels(1)
# pc.setGAMGSmooths(5)
ksp.setTolerances(1e-5, 1e-10)
ksp.setFromOptions()
ksp.solve(rhs, sol)

print("computation time: {}s".format(clock()-t))

In [ ]:
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(111, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY),)
sca = ax1.scatter3D(coords[:,0], coords[:,1], coords[:,2], c=sol.array)
fig.colorbar(sca)

# Refine

Refine the Y-axis

In [ ]:
def z_function(z):
    return 1.0 - np.exp(z)

mesh.refine(z_fn=z_function)

In [ ]:
mesh.boundary_condition('minZ', 1.0, flux=True)
mesh.boundary_condition('maxZ', 0.0, flux=False)

mesh.update_properties(k, H)

In [ ]:
for key in mesh.bc:
    print key, mesh.bc[key]['delta'], mesh.bc[key]['flux'], mesh.bc[key]['val']

print mesh.minZ, mesh.maxZ

In [ ]:
sol2 = mesh.solve()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(111, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY), )
sca = ax1.scatter3D(coords[:,0], coords[:,1], coords[:,2], c=sol2)
fig.colorbar(sca)
plt.savefig('rectilinear.png', bbox_inches='tight')

In [ ]:
H5_file = 'rectilinear-refined.h5'

mesh.save_mesh_to_hdf5(H5_file)
mesh.save_field_to_hdf5(H5_file, T=sol2, k=mesh.diffusivity, H=mesh.heat_sources)

conduction.generateXdmf(H5_file)

In [ ]:
from quagmire import tools as meshtools
meshtools.save_DM_to_hdf5(mesh.dm, 'rectilinear.h5')

vec = mesh.dm.createGlobalVector()
vec.setName('sol')
vec.setArray(sol)

ViewHDF5 = PETSc.Viewer()

ViewHDF5.createHDF5('rectilinear.h5', mode='a')
# ViewHDF5.pushFormat(PETSc.Viewer.Format.VTK_VTS)
ViewHDF5.view(obj=vec)
ViewHDF5.destroy()

# meshtools.generate_xdmf('rectilinear.h5')

In [ ]:
import h5py

f = h5py.File('rectilinear.h5', 'r+')

In [ ]:
f.create_group('topology')
topo = f['topology']

In [ ]:
topo.attrs.create('minCoord', np.array([minX, minY, minZ]))
topo.attrs.create('maxCoord', np.array([maxX, maxY, maxZ]))
topo.attrs.create('shape', np.array([nx, ny, nz]))
# topo.attrs.create('')

In [ ]:
mesh.dm.getSizes()

In [ ]:
f.close()

In [ ]:
# ViewHDF5.pushFormat(PETSc.Viewer.Format.VTK_VTR)
ViewHDF5.destroy()

In [ ]:
minCoord = np.array([minX, minY, minZ])